In [2]:
import nltk

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/vikaspuri/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vikaspuri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [11]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [12]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
47/47 [==============================] - 1s 27ms/step - loss: 2.2654 - accuracy: 0.1064
Epoch 2/200
47/47 [==============================] - 0s 2ms/step - loss: 2.0917 - accuracy: 0.2553
Epoch 3/200
47/47 [==============================] - 0s 1ms/step - loss: 2.0527 - accuracy: 0.2340
Epoch 4/200
47/47 [==============================] - 0s 2ms/step - loss: 1.8988 - accuracy: 0.4255
Epoch 5/200
47/47 [==============================] - 0s 2ms/step - loss: 1.8852 - accuracy: 0.3830
Epoch 6/200
47/47 [==============================] - 0s 1ms/step - loss: 1.7172 - accuracy: 0.4468
Epoch 7/200
47/47 [==============================] - 0s 1ms/step - loss: 1.5700 - accuracy: 0.4894
Epoch 8/200
47/47 [==============================] - 0s 2ms/step - loss: 1.3831 - accuracy: 0.5957
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 1.3069 - accuracy: 0.7021
Epoch 10/200
47/47 [==============================] - 0s 1ms/step - loss: 1.1754 - accuracy: 0.7234
Epoch 11

47/47 [==============================] - 0s 2ms/step - loss: 0.0772 - accuracy: 0.9787
Epoch 84/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0623 - accuracy: 0.9787
Epoch 85/200
47/47 [==============================] - 0s 4ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 4ms/step - loss: 0.1643 - accuracy: 0.9574
Epoch 87/200
47/47 [==============================] - 0s 1ms/step - loss: 0.1090 - accuracy: 0.9362
Epoch 88/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0353 - accuracy: 1.0000
Epoch 89/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9787
Epoch 90/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0752 - accuracy: 0.9574
Epoch 91/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0580 - accuracy: 0.9787
Epoch 92/200
47/47 [==============================] - 0s 3ms/step - loss: 0.0571 - accuracy: 1.0000
Epoch 93/200


47/47 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - ETA: 0s - loss: 0.0376 - accuracy: 1.00 - 0s 2ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0321 - accuracy: 0.9787
Epoch 166/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0356 - accuracy: 0.9787
Epoch 167/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0475 - accuracy: 0.9787
Epoch 168/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0234 - accuracy: 0.9787
Epoch 169/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 171/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 172/200
47/47 [==============================] - 0s 1ms/s